### 切木段问题 


![rod cut](./images/rod_cut.PNG)

In [154]:
prices = [0,1,5,8,9,10,17,17,20,24,30,33]
len(prices)

12

In [172]:
from collections import defaultdict
rodMaps = defaultdict(int,{i : prices[i] for i in range(len(prices))})

In [158]:
rodMaps

{0: 0,
 1: 1,
 2: 5,
 3: 8,
 4: 9,
 5: 10,
 6: 17,
 7: 17,
 8: 20,
 9: 24,
 10: 30,
 11: 33}

### 递归解决，没有cache

In [219]:
def getMaxValue(rodLen,solution):
    # if rod can not be cut
    if rodLen < 1:
        print("Sorry,your rod can not be cut")
        return 0
    if rodLen == 1 :
        return rodMaps[1]
    # or rod can be cut 
    maxV,leftV,rightV = rodMaps[rodLen],0,0
    tempsolution = (0,rodLen)
    for segment in range(1,rodLen):
        leftV = rodMaps[segment] if segment <= 11 else getMaxValue(segment,solution)
        rightV = rodMaps[rodLen-segment] if rodLen-segment <= 11 else getMaxValue(rodLen-segment,solution)
        if leftV + rightV > maxV:
            maxV = leftV + rightV
            tempsolution = (segment,rodLen-segment)
    solution.append((segment,rodLen-segment))
    return maxV

In [234]:
solution = []
%time v = getMaxValue(23,solution)
print(v)

Wall time: 710 ms
68


### 递归解决，带有cache

In [227]:
def getMaxValueWithCache(rodLen,solution,cache):
    # if rod can not be cut
    if rodLen < 1:
        print("Sorry,your rod can not be cut")
        return 0
    if rodLen == 1 :
        return rodMaps[1]
    # or rod can be cut 
    maxV,leftV,rightV = rodMaps[rodLen],0,0
    tempsolution = (0,rodLen)
    for segment in range(1,rodLen):
        # cut left
        if segment <= 11:
            leftV = rodMaps[segment]
        else:
            leftV = cache[segment] if cache[segment] else getMaxValueWithCache(segment,solution,cache)
            cache[segment] = leftV
        # cut right
        if rodLen-segment <=11:
            rightV = rodMaps[rodLen-segment]
        else:
            rightV = cache[rodLen-segment] if cache[rodLen-segment] else getMaxValueWithCache(rodLen-segment,solution,cache)
            cache[rodLen-segment] = rightV
        # select the best cut strategy
        if leftV + rightV > maxV:
            maxV = leftV + rightV
            tempsolution = (segment,rodLen-segment)
    solution.append(tempsolution)
    cache[rodLen] = maxV
    return maxV

In [228]:
from collections import defaultdict 
cache = defaultdict(int)

In [237]:
solution2 = []
%time getMaxValueWithCache(123,solution2,cache)

Wall time: 0 ns


369

In [236]:
solution2 = { item[0]+item[1] : (item[0],item[1]) for item in solution2}

In [218]:
solution2

{12: (2, 10),
 13: (2, 11),
 14: (3, 11),
 15: (2, 13),
 16: (6, 10),
 17: (6, 11),
 18: (2, 16),
 19: (2, 17),
 20: (10, 10),
 21: (10, 11),
 22: (11, 11),
 23: (2, 21)}

### 我太菜了，调了好久啊！😂
### 我自闭了，老师用5行代码就写出来了，我写的好繁琐😢

```python
def r(n):
    max_price, split_point = max(
        [(price[n], 0)] + [(r(i) + r(n-i), i) for i in range(1, n)], key=lambda x: x[0]
    )
    solution[n] = (split_point, n - split_point)
    return max_price
```

### 继续进阶！！！ 用装饰器特性写一个通用的缓存装饰器

In [327]:
from functools import wraps
def cache_decorator(func):
    from collections import defaultdict
    cache = defaultdict(int)
    def _wrap(rodLen):
        if cache[rodLen]: res = cache[rodLen]
        else:
            res = func(rodLen)
            cache[rodLen] = res
        return res
    return _wrap

In [328]:
@cache_decorator
def getMaxValueWithDecorator(rodLen):
    # if rod can not be cut
    if rodLen < 1:
        print("Sorry,your rod can not be cut")
        return 0
    if rodLen == 1 :
        return rodMaps[1]
    # or rod can be cut 
    maxV,leftV,rightV = rodMaps[rodLen],0,0
    for segment in range(1,rodLen):
        leftV = rodMaps[segment] if segment <= 11 else getMaxValueWithDecorator(segment)
        rightV = rodMaps[rodLen-segment] if rodLen-segment <= 11 else getMaxValueWithDecorator(rodLen-segment)
        if leftV + rightV > maxV:
            maxV = leftV + rightV
    return maxV

### 解决了一个疑问，装饰器的_wrap里cache缓存只是在func的外面进行缓存，为啥可以起效果
```python
getMaxValueWithDecorator = cache_decorator(getMaxValueWithDecorator)
%time getMaxValueWithDecorator(123)

func = cache_decorator(getMaxValueWithDecorator)
%time func(123)
```
### 经过实验，我的猜想得到证实，上面两种写法是不一样的，只有第一种写法可以起到缓存的效果，而@cache_decorator正是第一种写法的语法糖
### 这是为什么呢？
### 因为如果加入装饰器后，在赋值给自己，那么其实相当于内部函数名和调用函数名相同，这样就达到了内部递归的效果
### 但是如果是赋值给不相同的名字，那么就好比一个函数里调用另一个函数，这样cache就不能在内部递归，进而失效

In [323]:
%time getMaxValueWithDecorator(123)

Wall time: 0 ns


369

```python
from functools import wraps
original_price = [1, 5, 8, 9, 10, 17, 17, 20, 24, 30, 33]
price = defaultdict(int)
for i, p in enumerate(original_price):
    price[i+1] = p
def memo(func):
    cache = {}
    @wraps(func)
    def _wrap(n): ## ? *args, **kwargs
        if n in cache: result = cache[n]
        else:
            result = func(n)
            cache[n] = result
        return result
    return _wrap


@memo
def r(n):
    max_price, split_point = max(
        [(price[n], 0)] + [(r(i) + r(n-i), i) for i in range(1, n)], key=lambda x: x[0]
    )
    solution[n] = (split_point, n - split_point)
    
    return max_price
```